In [1]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

# import gensim.downloader as api

import codecs

%reload_ext tensorboard

SEED = 42
AUTOTUNE = tf.data.experimental.AUTOTUNE


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
print(path_to_file)
with open(path_to_file) as f:
    lines = f.read().splitlines()
for line in lines[:50]:
    print(line)

1122304/1115394 [==============================] - 0s 0us/step
C:\Users\edu\.keras\datasets\shakespeare.txt
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our 

In [3]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [48]:
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import preprocess_documents, preprocess_string
import more_itertools

doc = preprocess_documents(lines)
shake_ds = [x for x in doc if x]
# shake_ds = flatten(shake_ds)
# shake_ds = list(more_itertools.collapse(shake_ds))
shake_ds

[['citizen'],
 ['proce', 'hear', 'speak'],
 ['speak', 'speak'],
 ['citizen'],
 ['resolv', 'die', 'famish'],
 ['resolv', 'resolv'],
 ['citizen'],
 ['know', 'caiu', 'marciu', 'chief', 'enemi', 'peopl'],
 ['know', 'know'],
 ['citizen'],
 ['let', 'kill', 'corn', 'price'],
 ['verdict'],
 ['talk', 'let', 'awai', 'awai'],
 ['second', 'citizen'],
 ['word', 'good', 'citizen'],
 ['citizen'],
 ['account', 'poor', 'citizen', 'patrician', 'good'],
 ['author', 'surfeit', 'reliev'],
 ['yield', 'superflu'],
 ['wholesom', 'guess', 'reliev', 'human'],
 ['think', 'dear', 'lean'],
 ['afflict', 'object', 'miseri'],
 ['inventori', 'particularis', 'abund'],
 ['suffer', 'gain', 'let', 'reveng'],
 ['pike', 'er', 'rake', 'god', 'know'],
 ['speak', 'hunger', 'bread', 'thirst', 'reveng'],
 ['second', 'citizen'],
 ['proce', 'especi', 'caiu', 'marciu'],
 ['dog', 'commonalti'],
 ['second', 'citizen'],
 ['consid', 'servic', 'countri'],
 ['citizen'],
 ['content', 'good'],
 ['report', 'fort', 'pai', 'proud'],
 ['second

In [33]:
# string_shake_ds = ' '.join(element for element in shake_ds)
# # string_shake_ds
# text_file = open("gensim_shakespere.txt", "w")
# n = text_file.write(string_shake_ds)
# text_file.close()

In [49]:
from gensim import corpora

shakespeare_corpus = shake_ds
dictionary = corpora.Dictionary(shakespeare_corpus)
print(dictionary)

Dictionary(7512 unique tokens: ['citizen', 'hear', 'proce', 'speak', 'die']...)


In [47]:
vectors = w2v_model.wv
vectors.save_word2vec_format('gensim_will_s_model')

In [51]:
import gensim
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus('gensim_shakespere.txt'))
# test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [53]:
print(train_corpus[:2])

[TaggedDocument(words=['citizen', 'proce', 'hear', 'speak', 'speak', 'speak', 'citizen', 'resolv', 'die', 'famish', 'resolv', 'resolv', 'citizen', 'know', 'caiu', 'marciu', 'chief', 'enemi', 'peopl', 'know', 'know', 'citizen', 'let', 'kill', 'corn', 'price', 'verdict', 'talk', 'let', 'awai', 'awai', 'second', 'citizen', 'word', 'good', 'citizen', 'citizen', 'account', 'poor', 'citizen', 'patrician', 'good', 'author', 'surfeit', 'reliev', 'yield', 'superflu', 'wholesom', 'guess', 'reliev', 'human', 'think', 'dear', 'lean', 'afflict', 'object', 'miseri', 'inventori', 'particularis', 'abund', 'suffer', 'gain', 'let', 'reveng', 'pike', 'er', 'rake', 'god', 'know', 'speak', 'hunger', 'bread', 'thirst', 'reveng', 'second', 'citizen', 'proce', 'especi', 'caiu', 'marciu', 'dog', 'commonalti', 'second', 'citizen', 'consid', 'servic', 'countri', 'citizen', 'content', 'good', 'report', 'fort', 'pai', 'proud', 'second', 'citizen', 'nai', 'speak', 'malici', 'citizen', 'unto', 'hath', 'famous', 'end

In [55]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=128,window=2,seed=42,max_vocab_size=4096,epochs=20,hs=0,negative=4)

In [67]:
?gensim.models.doc2vec.Doc2Vec

In [57]:
model.build_vocab(train_corpus)

In [58]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [60]:
vector = model.infer_vector(['to','be','or','not','to','be'])
print(vector)

[ 2.00390816e-03  2.95447977e-03 -2.94135255e-03 -3.16232396e-03
  1.22490618e-03 -3.53305601e-04  1.25373481e-03 -3.21710994e-03
  2.71826982e-03 -2.46330164e-03 -3.41312028e-03  1.11663062e-03
  4.69319522e-04  4.84669581e-04  1.71031477e-03  2.96410080e-03
  8.58997460e-04  8.55559483e-04  8.80255830e-04 -3.57254944e-03
  1.65084004e-03  2.31034635e-03 -1.55116688e-03 -3.08412034e-03
 -1.84502522e-03 -1.33369071e-03 -8.66910210e-04  1.90212810e-03
 -1.33024226e-03  2.81944871e-04  1.36680668e-03  7.67852180e-05
 -3.79548827e-03 -2.05850392e-03 -1.30839786e-03 -1.59446313e-03
  6.27799425e-04  2.37977598e-03 -1.01670367e-03  3.26212822e-03
  2.62730755e-04  1.52638694e-03  2.24547228e-03  3.26564815e-03
 -1.01445988e-03  3.17280134e-03 -1.10724475e-04  3.04185133e-03
  3.52483848e-03 -1.80732645e-03  1.33941649e-03 -3.74594238e-05
  1.06161786e-03  3.58393695e-03 -1.70122087e-03  1.09186536e-03
  1.65065518e-03 -1.22588011e-03  2.72406219e-03 -9.81844729e-04
  9.26322769e-04 -2.81433

In [64]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

[('clarenc', 0.8435418009757996),
 ('heart', 0.839720606803894),
 ('propheci', 0.8392022252082825),
 ('winter', 0.8357893228530884),
 ('sun', 0.8326936960220337),
 ('dai', 0.8302764892578125),
 ('son', 0.8292179107666016),
 ('face', 0.8284216523170471),
 ('written', 0.8259100317955017),
 ('holi', 0.8256528377532959)]

In [66]:
vector = model.wv
vector.save_word2vec_format('will_s_model')